### Web Scraper of Trending Repositories on Github

In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import base64

In [62]:
topics_url = 'https://github.com/topics'

In [63]:
response = requests.get(topics_url)
content = response.text

In [64]:
content[:1000]

'\n\n<!DOCTYPE html>\n<html\n  lang="en"\n  \n  data-color-mode="auto" data-light-theme="light" data-dark-theme="dark"\n  data-a11y-animated-images="system" data-a11y-link-underlines="true"\n  \n  >\n\n\n\n  <head>\n    <meta charset="utf-8">\n  <link rel="dns-prefetch" href="https://github.githubassets.com">\n  <link rel="dns-prefetch" href="https://avatars.githubusercontent.com">\n  <link rel="dns-prefetch" href="https://github-cloud.s3.amazonaws.com">\n  <link rel="dns-prefetch" href="https://user-images.githubusercontent.com/">\n  <link rel="preconnect" href="https://github.githubassets.com" crossorigin>\n  <link rel="preconnect" href="https://avatars.githubusercontent.com">\n\n  \n\n  <link crossorigin="anonymous" media="all" rel="stylesheet" href="https://github.githubassets.com/assets/light-3e154969b9f9.css" /><link crossorigin="anonymous" media="all" rel="stylesheet" href="https://github.githubassets.com/assets/dark-9c5b7a476542.css" /><link data-color-theme="dark_dimmed" cross

In [65]:
with open("webpage.htmp", 'w') as f :
  f.write(content)

In [66]:
doc = BeautifulSoup(content,'html.parser')

In [67]:
topic_titles = doc.find_all('p', {'class' : 'f3 lh-condensed mb-0 mt-1 Link--primary'})
topic_titles

[<p class="f3 lh-condensed mb-0 mt-1 Link--primary">3D</p>,
 <p class="f3 lh-condensed mb-0 mt-1 Link--primary">Ajax</p>,
 <p class="f3 lh-condensed mb-0 mt-1 Link--primary">Algorithm</p>,
 <p class="f3 lh-condensed mb-0 mt-1 Link--primary">Amp</p>,
 <p class="f3 lh-condensed mb-0 mt-1 Link--primary">Android</p>,
 <p class="f3 lh-condensed mb-0 mt-1 Link--primary">Angular</p>,
 <p class="f3 lh-condensed mb-0 mt-1 Link--primary">Ansible</p>,
 <p class="f3 lh-condensed mb-0 mt-1 Link--primary">API</p>,
 <p class="f3 lh-condensed mb-0 mt-1 Link--primary">Arduino</p>,
 <p class="f3 lh-condensed mb-0 mt-1 Link--primary">ASP.NET</p>,
 <p class="f3 lh-condensed mb-0 mt-1 Link--primary">Awesome Lists</p>,
 <p class="f3 lh-condensed mb-0 mt-1 Link--primary">Amazon Web Services</p>,
 <p class="f3 lh-condensed mb-0 mt-1 Link--primary">Azure</p>,
 <p class="f3 lh-condensed mb-0 mt-1 Link--primary">Babel</p>,
 <p class="f3 lh-condensed mb-0 mt-1 Link--primary">Bash</p>,
 <p class="f3 lh-condensed m

In [68]:
a = topic_titles[0].parent.parent.find_all('a',{'class':'no-underline flex-grow-0'}, href=True)
a[0]['href']

'/topics/3d'

In [69]:
topic_titles_ = []
topic_urls = []
for tag in topic_titles :
  topic_titles_.append(tag.text)
  topic_urls.append("https://github.com"+tag.parent.parent.find_all('a',{'class':'no-underline flex-grow-0'}, href=True)[0]['href'])
topic_urls


['https://github.com/topics/3d',
 'https://github.com/topics/ajax',
 'https://github.com/topics/algorithm',
 'https://github.com/topics/amphp',
 'https://github.com/topics/android',
 'https://github.com/topics/angular',
 'https://github.com/topics/ansible',
 'https://github.com/topics/api',
 'https://github.com/topics/arduino',
 'https://github.com/topics/aspnet',
 'https://github.com/topics/awesome',
 'https://github.com/topics/aws',
 'https://github.com/topics/azure',
 'https://github.com/topics/babel',
 'https://github.com/topics/bash',
 'https://github.com/topics/bitcoin',
 'https://github.com/topics/bootstrap',
 'https://github.com/topics/bot',
 'https://github.com/topics/c',
 'https://github.com/topics/chrome',
 'https://github.com/topics/chrome-extension',
 'https://github.com/topics/cli',
 'https://github.com/topics/clojure',
 'https://github.com/topics/code-quality',
 'https://github.com/topics/code-review',
 'https://github.com/topics/compiler',
 'https://github.com/topics/co

In [70]:
df = pd.DataFrame({'topic_title': topic_titles_, 'topic_url': topic_urls})

In [71]:
df.head()

,topic_title,topic_url
0,3D,https://github.com/topics/3d
1,Ajax,https://github.com/topics/ajax
2,Algorithm,https://github.com/topics/algorithm
3,Amp,https://github.com/topics/amphp
4,Android,https://github.com/topics/android


In [72]:
def get_info_from_topic(url) :
  response = requests.get(url).text
  content = BeautifulSoup(response,'html.parser')
  usernames = []
  repos = []
  repo_urls = []
  stars = []
  repositories = content.find_all('h3',{'class': "f3 color-fg-muted text-normal lh-condensed"})
  stars_raw = content.find_all('span', {'id' : "repo-stars-counter-star"})
  for i in range(len(repositories)) :
    repo = repositories[i]
    usernames.append(repo.find_all('a')[0].text.strip())
    repos.append(repo.find_all('a')[1].text.strip())
    repo_urls.append("https://github.com"+repo.find_all('a')[1]['href'])
    star = stars_raw[i].text.strip()
    if star[-1]=='k' :
      star = star[:-1]
      star = float(star)
      star = int(star*1000)
    else : star = int(star)
    stars.append(star)
  return {
      "usernames" : usernames,
      "repos" : repos,
      "stars" : stars,
      "repo_urls" : repo_urls,
  }


In [74]:
df = pd.DataFrame(get_info_from_topic("https://github.com/topics/Android"))
df.head()

,usernames,repos,stars,repo_urls
0,flutter,flutter,165000,https://github.com/flutter/flutter
1,facebook,react-native,119000,https://github.com/facebook/react-native
2,justjavac,free-programming-books-zh_CN,111000,https://github.com/justjavac/free-programming-...
3,Genymobile,scrcpy,110000,https://github.com/Genymobile/scrcpy
4,Hack-with-Github,Awesome-Hacking,83900,https://github.com/Hack-with-Github/Awesome-Ha...


#save the data in to csv

In [75]:
for i in range(len(topic_titles_)) :
  df = pd.DataFrame(get_info_from_topic(topic_urls[i]))
  df.to_csv(topic_titles_[i])

scrap the readme file of repo.

In [76]:
import requests
from bs4 import BeautifulSoup

# URL of the README page
url = "https://github.com/bumptech/glide"

# Fetch the README page
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Extract the title (usually in <h1> or <h2>)
title = soup.find('h1').get_text(strip=True)

# Extract the description (first <p> after title or within a specific section)
description = soup.find('p').get_text(strip=True)

# Extract all paragraphs (for main content)
content_paragraphs = soup.find_all('p')
content = [para.get_text(strip=True) for para in content_paragraphs]

# Extract links from the content
links = []
for link in soup.find_all('a', href=True):
    links.append((link.get_text(strip=True), link['href']))



In [77]:
# Display the extracted content
print("Title:", title)



Title: Search code, repositories, users, issues, pull requests...


In [78]:
print("Content:", content)


Content: ['We read every piece of feedback, and take your input very seriously.', 'To see all available qualifiers, see ourdocumentation.', 'An image loading and caching library for Android focused on smooth scrolling', "|View Glide's documentation|简体中文文档|Report an issue with Glide", 'Glide is a fast and efficient open source media management and image loading framework for Android that wraps media\ndecoding, memory and disk caching, and resource pooling into a simple and easy to use interface.', '', "Glide supports fetching, decoding, and displaying video stills, images, and animated GIFs. Glide includes a flexible API\nthat allows developers to plug in to almost any network stack. By default Glide uses a customHttpUrlConnectionbased\nstack, but also includes utility libraries plug in to Google's Volley project or Square's OkHttp library instead.", "Glide's primary focus is on making scrolling any kind of a list of images as smooth and fast as possible, but Glide is\nalso effective fo

In [79]:
print("Links:", links)

Links: [('Skip to content', '#start-of-content'), ('', '/'), ('Sign in', '/login?return_to=https%3A%2F%2Fgithub.com%2Fbumptech%2Fglide'), ('GitHub CopilotWrite better code with AI', 'https://github.com/features/copilot'), ('SecurityFind and fix vulnerabilities', 'https://github.com/features/security'), ('ActionsAutomate any workflow', 'https://github.com/features/actions'), ('CodespacesInstant dev environments', 'https://github.com/features/codespaces'), ('IssuesPlan and track work', 'https://github.com/features/issues'), ('Code ReviewManage code changes', 'https://github.com/features/code-review'), ('DiscussionsCollaborate outside of code', 'https://github.com/features/discussions'), ('Code SearchFind more, search less', 'https://github.com/features/code-search'), ('All features', 'https://github.com/features'), ('Documentation', 'https://docs.github.com'), ('GitHub Skills', 'https://skills.github.com'), ('Blog', 'https://github.blog'), ('Enterprise', 'https://github.com/enterprise'),

merge both logic

In [3]:





def scrape_readme(url):
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code != 200:
        print(f"Failed to fetch {url}: {response.status_code}")
        return None, "No description found", [], []

    soup = BeautifulSoup(response.text, 'html.parser')

    # Extract the title (usually in <h1>)
    title_tag = soup.find('h1')
    title = title_tag.get_text(strip=True) if title_tag else "No title found"

    # Find the first <p> after the title
    content_paragraphs = soup.find_all('p')

    # Extract the description (first <p> after title or within a specific section)
    description = soup.find('p').get_text(strip=True)

    # Extract all paragraphs (for main content)

    content = [para.get_text(strip=True) for para in content_paragraphs]

    # Extract links from the README content
    links = []
    for link in soup.find_all('a', href=True):
        links.append((link.get_text(strip=True), link['href']))

    return title, description, content, links






def scrape_trending_repositories(limit=100):  # Set a default limit
    topics_url = 'https://github.com/topics'
    response = requests.get(topics_url)
    content = response.text

    doc = BeautifulSoup(content, 'html.parser')
    topic_titles = doc.find_all('p', {'class': 'f3 lh-condensed mb-0 mt-1 Link--primary'})

    topic_titles_ = []
    topic_urls = []

    for tag in topic_titles:
        topic_titles_.append(tag.text)
        topic_urls.append("https://github.com" + tag.parent.parent.find_all('a', {'class': 'no-underline flex-grow-0'}, href=True)[0]['href'])

    df_topics = pd.DataFrame({'topic_title': topic_titles_, 'topic_url': topic_urls})

    all_repo_data = []

    # Loop through each topic URL to get repository info
    for topic_url in topic_urls:
        response = requests.get(topic_url)
        content = BeautifulSoup(response.text, 'html.parser')
        repositories = content.find_all('h3', {'class': "f3 color-fg-muted text-normal lh-condensed"})

        for repo in repositories:
            if len(all_repo_data) >= limit:  # Check if the limit is reached
                break

            user = repo.find_all('a')[0].text.strip()
            repo_name = repo.find_all('a')[1].text.strip()
            repo_url = "https://github.com" + repo.find_all('a')[1]['href']

            # Get the stars (if available)
            star_tag = repo.find_next('span', {'id': 'repo-stars-counter-star'})
            if star_tag:
                star_text = star_tag.text.strip().replace(',', '')
                if 'k' in star_text:
                    star_text = star_text.replace('k', '')  # Remove the 'k'
                    stars = int(float(star_text) * 1000)  # Convert to integer after multiplying by 1000
                else:
                    stars = int(float(star_text))  # Convert to integer directly
            else:
                stars = 0

            # Scrape the README content for the repository
            readme_title, readme_description, readme_content, readme_links = scrape_readme(repo_url)

            # Collect all data in a dictionary
            repo_data = {
                'username': user,
                'repo': repo_name,
                'stars': stars,
                'repo_url': repo_url,
                  'readme_content': readme_content,


                'readme_links': readme_links
            }
            all_repo_data.append(repo_data)

    return all_repo_data


In [117]:
df_repos = pd.DataFrame(scrape_trending_repositories())
df_repos.head()

,username,repo,stars,repo_url,readme_content,readme_description,readme_links
0,mrdoob,three.js,102000,https://github.com/mrdoob/three.js,"[We read every piece of feedback, and take you...","We read every piece of feedback, and take your...","[(Skip to content, #start-of-content), (, /), ..."
1,pmndrs,react-three-fiber,27300,https://github.com/pmndrs/react-three-fiber,"[We read every piece of feedback, and take you...","We read every piece of feedback, and take your...","[(Skip to content, #start-of-content), (, /), ..."
2,libgdx,libgdx,23300,https://github.com/libgdx/libgdx,"[We read every piece of feedback, and take you...","We read every piece of feedback, and take your...","[(Skip to content, #start-of-content), (, /), ..."
3,BabylonJS,Babylon.js,23200,https://github.com/BabylonJS/Babylon.js,"[We read every piece of feedback, and take you...","We read every piece of feedback, and take your...","[(Skip to content, #start-of-content), (, /), ..."
4,ssloy,tinyrenderer,20400,https://github.com/ssloy/tinyrenderer,"[We read every piece of feedback, and take you...","We read every piece of feedback, and take your...","[(Skip to content, #start-of-content), (, /), ..."


summarization model


In [4]:
from transformers import pipeline

# Load the summarization model
summarization_model = pipeline("summarization", model="facebook/bart-large-cnn")

# Sample README content as a single string (you can combine from your list)
readme_content = """We read every piece of feedback, and take your input very seriously.
The aim of the project is to create an easy-to-use, lightweight, cross-browser, general-purpose 3D library.
The current builds only include a WebGL renderer but WebGPU (experimental), SVG, and CSS3D renderers are also available as addons.
This code creates a scene, a camera, and a geometric cube, and it adds the cube to the scene.
It then creates a WebGL renderer for the scene and camera, and it adds that viewport to the document.body element.
Finally, it animates the cube within the scene for the camera.
If everything goes well, you should see this.
Cloning the repo with all its history results in a ~2 GB download.
If you don't need the whole history you can use the depth parameter to significantly reduce download size.
JavaScript 3D Library."""

# Function to summarize the README content
def summarize_readme(text):
    # Summarize the text using the summarization model
    summary = summarization_model(text, max_length=300, min_length=30, do_sample=False)
    return summary[0]['summary_text']  # Extract the summary text

# Call the summarization function
summary = summarize_readme(readme_content)
print("Summary:")
print(summary)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Your max_length is set to 300, but your input_length is only 201. Since this is a summariza

Summary:
The aim of the project is to create an easy-to-use, lightweight, cross-browser, general-purpose 3D library. Current builds only include a WebGL renderer but WebGPU (experimental), SVG, and CSS3D renderers are also available as addons.


In [121]:
#merge this logic

In [5]:
# Function to summarize the README content
def summarize_readme(content):
    if not content:
        return "No content to summarize."

    # Join all paragraphs to form a single string
    full_text = " ".join(content)

    # Check if the length of the full text exceeds a reasonable length
    max_length = 1024  # Set a limit for the summarization model
    if len(full_text) > max_length:
        full_text = full_text[:max_length]  # Truncate the text

    try:
        # Summarize the text using the summarization model
        summary = summarization_model(full_text, max_length=130, min_length=30, do_sample=False)
        return summary[0]['summary_text']  # Extract the summary text
    except Exception as e:
        print(f"Error during summarization: {e}")
        return "Error occurred during summarization."

# The rest of your existing code follows here...


In [6]:
def scrape_readme(url):
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code != 200:
        print(f"Failed to fetch {url}: {response.status_code}")
        return None, "No description found", [], []

    soup = BeautifulSoup(response.text, 'html.parser')

    # Extract the title (usually in <h1>)
    title_tag = soup.find('h1')
    title = title_tag.get_text(strip=True) if title_tag else "No title found"

    # Find the first <p> after the title
    content_paragraphs = soup.find_all('p')

    # Extract the description (first <p> after title)
    description = soup.find('p').get_text(strip=True) if soup.find('p') else "No description found"

    # Extract all paragraphs for main content
    content = [para.get_text(strip=True) for para in content_paragraphs]

    # Extract links from the README content
    links = []
    for link in soup.find_all('a', href=True):
        links.append((link.get_text(strip=True), link['href']))

    return title, description, content, links



In [8]:
# Scrape the trending repositories and extract README summaries
def scrape_trending_repositories(limit=10):  # Set a default limit
    topics_url = 'https://github.com/topics'
    response = requests.get(topics_url)
    content = response.text

    doc = BeautifulSoup(content, 'html.parser')
    topic_titles = doc.find_all('p', {'class': 'f3 lh-condensed mb-0 mt-1 Link--primary'})

    topic_titles_ = []
    topic_urls = []

    for tag in topic_titles:
        topic_titles_.append(tag.text)
        topic_urls.append("https://github.com" + tag.parent.parent.find_all('a', {'class': 'no-underline flex-grow-0'}, href=True)[0]['href'])

    all_repo_data = []

    # Loop through each topic URL to get repository info
    for topic_url in topic_urls:
        response = requests.get(topic_url)
        content = BeautifulSoup(response.text, 'html.parser')
        repositories = content.find_all('h3', {'class': "f3 color-fg-muted text-normal lh-condensed"})

        for repo in repositories:
            if len(all_repo_data) >= limit:  # Check if the limit is reached
                break

            user = repo.find_all('a')[0].text.strip()
            repo_name = repo.find_all('a')[1].text.strip()
            repo_url = "https://github.com" + repo.find_all('a')[1]['href']

            # Get the stars (if available)
            star_tag = repo.find_next('span', {'id': 'repo-stars-counter-star'})
            if star_tag:
                star_text = star_tag.text.strip().replace(',', '')
                if 'k' in star_text:
                    star_text = star_text.replace('k', '')  # Remove the 'k'
                    stars = int(float(star_text) * 1000)  # Convert to integer after multiplying by 1000
                else:
                    stars = int(float(star_text))  # Convert directly to integer
            else:
                stars = 0

            # Scrape the README content for the repository
            readme_title, readme_description, readme_content, readme_links = scrape_readme(repo_url)

            # Summarize the README content
            readme_summary = summarize_readme(readme_content)

            # Collect all data in a dictionary
            repo_data = {
                'username': user,
                'repo': repo_name,
                'stars': stars,
                'repo_url': repo_url,
                'readme_content': readme_content,
                'readme_summary': readme_summary,
                'readme_links': readme_links
            }
            all_repo_data.append(repo_data)

    return all_repo_data

# Execute the scraping and save to CSV
repo_data = scrape_trending_repositories()
df_repos = pd.DataFrame(repo_data)
df_repos.to_csv('trending_repositories_with_readme_summary.csv', index=False)

Your max_length is set to 130, but your input_length is only 125. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=62)


In [9]:
df_repos.head()

,username,repo,stars,repo_url,readme_content,readme_summary,readme_links
0,mrdoob,three.js,102000,https://github.com/mrdoob/three.js,"[We read every piece of feedback, and take you...",The aim of the project is to create an easy-to...,"[(Skip to content, #start-of-content), (, /), ..."
1,pmndrs,react-three-fiber,27300,https://github.com/pmndrs/react-three-fiber,"[We read every piece of feedback, and take you...",react-three-fiber is aReact rendererfor three...,"[(Skip to content, #start-of-content), (, /), ..."
2,libgdx,libgdx,23300,https://github.com/libgdx/libgdx,"[We read every piece of feedback, and take you...",libGDX is a cross-platform Java game developm...,"[(Skip to content, #start-of-content), (, /), ..."
3,BabylonJS,Babylon.js,23200,https://github.com/BabylonJS/Babylon.js,"[We read every piece of feedback, and take you...",WARNING: The CDN should not be used in product...,"[(Skip to content, #start-of-content), (, /), ..."
4,ssloy,tinyrenderer,20400,https://github.com/ssloy/tinyrenderer,"[We read every piece of feedback, and take you...",Gitpod is a free online dev evironment for Git...,"[(Skip to content, #start-of-content), (, /), ..."
